Point is to define a metaclass which does all the initialization, so instead of 
```python
class A:
    def __init__(self, a, b, c, d, e, f, g):
        self.a
        self.b
        self.c
        self.d
        self.e
        self.f
        self.g
```

that boiler could be done via the metaclass

In [182]:
class InitMeta(type):
    '''MetaClass to reduce boilerplate
    
    Example usage:
    
        Instea of defining a clas initializer with explicity initialization
        class A:
            def __init__(self, a, b, c, d):
                self.a = a
                self.b = b
                self.c = c
                self.d = d

        specifying the metaclass as InitMeta modifies the original init
            adding class-initialization boilerplate
        class A(metaclass=InitMeta):
            def __init__(self, a, b, c, d):

                print(self.a) # This works even though self.a was not explicitly set

        This reduces the clutter when multiple attributes are passed in to the class constructor

        Raises:
            RuntimeError: if __init__ is not defined 
    '''



    import inspect

    def __new__(cls, name, bases, attributes):
        if not (cls_init := attributes.get('__init__', None)):
            raise RuntimeError('__init__ must be specified')

        init_args = list(InitMeta.inspect.signature(cls_init).parameters.keys())[1:]

        def meta_init(self, *args, **kwargs):
            # set kwargs first, else non-kwarg is overritten by get() returning None
            for arg in init_args:
                setattr(self, arg, kwargs.get(arg))

            for arg_name, arg in zip(init_args, args):
                setattr(self, arg_name, arg)


            cls_init(self, *args, **kwargs)

        attributes['__init__'] = meta_init

        return super(InitMeta, cls).__new__(cls, name, bases, attributes)


class A(metaclass=InitMeta):
    def __init__(self, a, b):

        print(self.a)

In [183]:
a = A(1, 2)
b = A(a=4, b=5)

1
4


In [179]:
a.a, a.b, b.a, b.b

(1, 2, 4, 5)